In [1]:
from fingerprint import *

In [8]:
# set path to a file or directory to add it to the database

path = './test_songs/Everglow/'
add_song(path)

skipping 03 D+1.m4a
skipping 02 Bon Bon Chocolat.m4a
skipping cover.jpg
skipping 01 Moon.m4a
